In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import parse_raw_to_csv
import os
import numpy as np

df = pd.DataFrame()

N = [1,2,5,8,13]
n = [1,4,9,16,25]

for ii, i in enumerate(N):
    df_tmp = pd.DataFrame()
    for f in os.listdir("../../experiments/%d_fpga" % i):
        if any(f.endswith(e) for e in [".txt",".out"]) and any(f.startswith(s) for s in ["hpl"]) and not "b8" in f:
            df_parsed = parse_raw_to_csv.parse_file_or_folder(os.path.join("../../experiments/%d_fpga" % i, f), parse_raw_to_csv.parse_map.values())
            if df_parsed.empty:
                continue
            if "pcie" in f:
                df_parsed["type"] = "PCIE"
                df_parsed["norm_flops"] = float(df_parsed["lu_gflops"]) / (233.4 * 5 * (n[ii])) * 100
            else:
                df_parsed["type"] = "IEC"
                df_parsed["norm_flops"] = float(df_parsed["lu_gflops"]) / (244 * 5 * (n[ii])) * 100
            df_tmp = df_tmp.append(df_parsed)
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df = df.append(df_tmp)

for ii, i in enumerate(N):
    df_tmp = pd.DataFrame()
    for f in os.listdir("../../experiments/%d_fpga" % i):
        if any(f.endswith(e) for e in [".txt",".out"]) and any(f.startswith(s) for s in ["hpl"]) and "b8" in f:
            df_parsed = parse_raw_to_csv.parse_file_or_folder(os.path.join("../../experiments/%d_fpga" % i, f), parse_raw_to_csv.parse_map.values())
            if df_parsed.empty:
                continue
            df_parsed["type"] = "PCIE (b=8)"
            df_parsed["norm_flops"] = float(df_parsed["lu_gflops"]) / (280 * 2 * (n[ii])) * 100
            df_tmp = df_tmp.append(df_parsed)
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df = df.append(df_tmp)

for ii, i in enumerate([1,2]):
    df_tmp = pd.DataFrame()
    for f in os.listdir("../../experiments/%d_fpga/u280" % i):
        if any(f.endswith(e) for e in [".txt",".out"]) and any(f.startswith(s) for s in ["hpl"]):
            if df_parsed.empty:
                continue
            df_parsed = parse_raw_to_csv.parse_file_or_folder(os.path.join("../../experiments/%d_fpga/u280" % i, f), parse_raw_to_csv.parse_map.values())
            df_parsed["type"] = "PCIE"
            df_parsed["norm_flops"] = float(df_parsed["lu_gflops"]) / (253 * 2 * (n[ii])) * 100
            df_tmp = df_tmp.append(df_parsed)
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df = df.append(df_tmp)


df["benchmark"] = df.index
df = df.apply(pd.to_numeric, errors='ignore')
df = df.reset_index()


df

In [ ]:
df

In [ ]:

df_strongt = df
df_strongt["total_size"] = df_strongt["size"]*np.sqrt(df_strongt["n"])
df_strong = df_strongt[df_strongt["total_size"].isin([48 * 512, 25600])]
df_weak = df_strongt[df_strongt["total_size"] == 48 *np.sqrt(df_strongt["n"]) * 512]
df_strong

In [ ]:
import re
df_cpu_strong = pd.DataFrame()

regex = "WC00C2R2\\s+(?P<size>\\d+)\\s+\\d+\\s+\\d+\\s+\\d+\\s+(?P<lu_best_time>\\d+\.\\d+)\\s+(?P<lu_gflops>(\d|\.|\+|-|e)+)"

for ii, i in enumerate(N):
    df_tmp = pd.DataFrame()
    try:
        with open("../../experiments/cpu_hpl/strong_scaling/%d_cpu/xhpl_intel64_static_outputs.txt" % i) as f:
                content = f.read()
                res = re.search(regex, content)
                if res is not None:
                    df_tmp = df_tmp.append(res.groupdict(), ignore_index=True)
    except:
        pass
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df_cpu_strong = df_cpu_strong.append(df_tmp)


df_cpu_strong["benchmark"] = "MP LINPACK"
df_cpu_strong = df_cpu_strong.apply(pd.to_numeric, errors='ignore')
df_cpu_strong = df_cpu_strong.reset_index()

df_cpu_weak = pd.DataFrame()

for ii, i in enumerate(N):
    df_tmp = pd.DataFrame()
    with open("../../experiments/cpu_hpl/weak_scaling/%d_cpu/xhpl_intel64_static_outputs.txt" % i) as f:
            content = f.read()
            res = re.search(regex, content)
            if res is not None:
                df_tmp = df_tmp.append(res.groupdict(), ignore_index=True)
    df_tmp["N"] = i
    df_tmp["n"] = n[ii]
    df_cpu_weak = df_cpu_weak.append(df_tmp)


df_cpu_weak["benchmark"] = "MP LINPACK"
df_cpu_weak= df_cpu_weak.apply(pd.to_numeric, errors='ignore')
df_cpu_weak = df_cpu_weak.reset_index()

df_cpu_weak

In [ ]:
import math
b=512
g=8
f=270e6
f_c = 157e6
t_mm = ((b ** 2)/(g*f)*2 + (b/g) ** 3 / f)
#       load and store block   forward rows and cols    update matrix
t_lu = (b ** 2)/(g*f)*2 + sum((b/g)/f_c + (b * (b - g * int(i/g)))/(g ** 2) / f for i in range(b))

def perf_last(m, torus_width=1, time=False, r=5):
    #LU time at the very beginning of the calculation and at the very end
    lu_total = max(m - 1, 1) * t_lu
    def lu_runtime(i):
        return max(math.ceil(i/torus_width) - 1,1) * t_lu #+ max(math.ceil(i/torus_width) - 2,0) * (700*b)/f
    def mm_runtime(i):
        remaining_blocks = math.ceil(i/torus_width)
        return math.ceil((remaining_blocks ** 2) / r) * t_mm
    main_calc_time = sum(max(math.ceil(math.ceil(max(i + i - 1, 0) / torus_width) / r) * t_mm + lu_runtime(i), mm_runtime(i))  for i in range(1,m * torus_width))
    exe_time = lu_total + main_calc_time
    gflops = (2*((m*torus_width*512) ** 3)/3)/ exe_time * 1.0e-9
    if time:
        return exe_time
    else:
        return gflops

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, hpl = plt.subplots(1)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("xilinx")]["n"]) - 0.2, df_weak[df_weak["device"].str.contains("xilinx")]["norm_flops"], label="Xilinx Alveo U280", color="C0", width=0.2)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("p520_hpc")]["n"]) - 0.0, df_weak[df_weak["device"].str.contains("p520_hpc")]["norm_flops"], label="Nallatech 520N HPC", color="C1", width=0.2)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("p520_max")]["n"]) + 0.2, df_weak[df_weak["device"].str.contains("p520_max")]["norm_flops"], label="Nallatech 520N MAX", color="C2", width=0.2)

for t in range(2,6):
    hpl.bar(t-0.2, df_weak[(df_weak["size"] == np.ceil(48) * 512) & (df_weak["device"].str.contains("xilinx")) & (df_weak["n"] == 1)]["norm_flops"], color="C0", alpha=0.6, width=0.2)
    hpl.bar(t+0.2, df_weak[(df_weak["size"] == np.ceil(48) * 512) & (df_weak["device"].str.contains("p520_max")) & (df_weak["n"] == 1)]["norm_flops"], color="C2", alpha=0.6, width=0.2)
    #hpl.bar(t+0.0, df_weak[(df_weak["device"].str.contains("p520_hpc")) & (df_weak["n"] == 1)]["norm_flops"], color="C1", alpha=0.6, width=0.2)

#hpl.bar(np.array(range(2,5))+0.4, [68.6, 68.1, 67.8], color="C2", alpha=0.6, width=0.2)

hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("norm. GFLOP/s")
hpl.set_ylim((0.0,72.0))
hpl.legend(loc="lower right", bbox_to_anchor=(1,0))
fig.savefig("hpl_torus_norm.pdf")

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, hpl = plt.subplots(1)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("xilinx")]["n"]) - 0.2, df_weak[df_weak["device"].str.contains("xilinx")]["total_gflops"] * 1.0e-3, label="Xilinx Alveo U280", color="C0", width=0.2)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("p520_hpc")]["n"]) - 0.0, df_weak[df_weak["device"].str.contains("p520_hpc")]["total_gflops"] * 1.0e-3, label="Nallatech 520N HPC", color="C1", width=0.2)
hpl.bar(np.sqrt(df_weak[df_weak["device"].str.contains("p520_max")]["n"]) + 0.2, df_weak[df_weak["device"].str.contains("p520_max")]["total_gflops"] * 1.0e-3, label="Nallatech 520N MAX", color="C2", width=0.2)

for t in range(2,6):
    hpl.bar(t-0.2, df_weak[(df_weak["size"] == np.ceil(48) * 512) & (df_weak["device"].str.contains("xilinx")) & (df_weak["n"] == 1)]["total_gflops"] * 1.0e-3 * (t ** 2), color="C0", alpha=0.6, width=0.2)
    hpl.bar(t+0.2, df_weak[(df_weak["size"] == np.ceil(48) * 512) & (df_weak["device"].str.contains("p520_max")) & (df_weak["n"] == 1)]["total_gflops"] * 1.0e-3 * (t ** 2), color="C2", alpha=0.6, width=0.2)
    #hpl.bar(t+0.0, df_weak[(df_weak["device"].str.contains("p520_hpc")) & (df_weak["n"] == 1)]["norm_flops"], color="C1", alpha=0.6, width=0.2)

hpl.bar(np.array(range(2,5))+0.4, np.array([68.6, 68.1, 67.8])  * 1.0e-3, color="C2", alpha=0.6, width=0.2)


hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("TFLOP/s")
#hpl.set_ylim((0.0,72.0))
hpl.legend(loc="upper left", bbox_to_anchor=(0,1))
fig.savefig("hpl_torus.pdf")

In [ ]:
df_scaling = pd.read_csv("hpl_scaling.csv")

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, hpl = plt.subplots(1)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("xilinx")]["n"]) - 0.2, df_strong[df_strong["device"].str.contains("xilinx")]["norm_flops"], label="Xilinx Alveo U280", color="C0", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_hpc")]["n"]) - 0.0, df_strong[df_strong["device"].str.contains("p520_hpc")]["norm_flops"], label="Nallatech 520N HPC", color="C1", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_max")]["n"]) + 0.2, df_strong[df_strong["device"].str.contains("p520_max")]["norm_flops"], label="Nallatech 520N MAX", color="C2", width=0.2)

for t in range(2,6):
    hpl.bar(t-0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("xilinx"))]["norm_flops"], color="C0", alpha=0.6, width=0.2)
for t in range(1,6):
    hpl.bar(t+0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("p520_max"))]["norm_flops"], color="C2", alpha=0.6, width=0.2)
    hpl.bar(t+0.0, df_scaling[(df_scaling["m"] == np.ceil(48/t)) & (df_scaling["size"]/np.ceil(48/t) == 512) & (df_scaling["device"].str.contains("p520_hpc"))]["norm_flops"], color="C1", alpha=0.6, width=0.2)


hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("norm. GFLOP/s")
hpl.set_ylim((0.0,72.0))
hpl.legend(loc="upper right", bbox_to_anchor=(1,1))
fig.savefig("hpl_torus_norm.pdf")

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, hpl = plt.subplots(1)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("xilinx")]["n"]) - 0.2, df_strong[df_strong["device"].str.contains("xilinx")]["total_gflops"], label="Xilinx Alveo U280", color="C0", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_hpc")]["n"]) - 0.0, df_strong[df_strong["device"].str.contains("p520_hpc")]["total_gflops"], label="Nallatech 520N HPC", color="C1", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_max")]["n"]) + 0.2, df_strong[df_strong["device"].str.contains("p520_max")]["total_gflops"], label="Nallatech 520N MAX", color="C2", width=0.2)

for t in range(2,6):
    hpl.bar(t-0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("xilinx"))]["total_gflops"] * (t ** 2), color="C0", alpha=0.6, width=0.2)
for t in range(1,6):
    hpl.bar(t+0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("p520_max"))]["total_gflops"] * (t ** 2), color="C2", alpha=0.6, width=0.2)
    hpl.bar(t+0.0, df_scaling[(df_scaling["m"] == np.ceil(48/t)) & (df_scaling["size"]/np.ceil(48/t) == 512) & (df_scaling["device"].str.contains("p520_hpc"))]["total_gflops"] * (t ** 2), color="C1", alpha=0.6, width=0.2)



hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("norm. GFLOP/s")
#hpl.set_ylim((0.0,72.0))
hpl.legend(loc="upper left", bbox_to_anchor=(0,1))
fig.savefig("hpl_torus_strong.pdf")

In [ ]:
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

fig, hpl = plt.subplots(1)
print(df_strong[df_strong["device"].str.contains("p520_hpc")]["norm_flops"] /  df_strong[(df_strong["device"].str.contains("p520_hpc")) & (df_strong["n"] == 1)]["norm_flops"].values[0])
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("xilinx")]["n"]) - 0.2, df_strong[df_strong["device"].str.contains("xilinx")]["norm_flops"] /  df_strong[(df_strong["device"].str.contains("xilinx")) & (df_strong["n"] == 1)]["norm_flops"].values[0], label="Xilinx Alveo U280", color="C0", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_hpc")]["n"]) - 0.0, df_strong[df_strong["device"].str.contains("p520_hpc")]["norm_flops"]  /  df_strong[(df_strong["device"].str.contains("p520_hpc")) & (df_strong["n"] == 1)]["norm_flops"].values[0], label="Nallatech 520N HPC", color="C1", width=0.2)
hpl.bar(np.sqrt(df_strong[df_strong["device"].str.contains("p520_max")]["n"]) + 0.2, df_strong[df_strong["device"].str.contains("p520_max")]["norm_flops"]  /  df_strong[(df_strong["device"].str.contains("p520_max")) & (df_strong["n"] == 1)]["norm_flops"].values[0], label="Nallatech 520N MAX", color="C2", width=0.2)

for t in range(2,6):
    hpl.bar(t-0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("xilinx"))]["norm_flops"]  /  df_strong[(df_strong["device"].str.contains("xilinx")) & (df_strong["n"] == 1)]["norm_flops"].values[0], color="C0", alpha=0.6, width=0.2)
for t in range(1,6):
    hpl.bar(t+0.2, df_scaling[(df_scaling["size"] == np.ceil(48/t) * 512) & (df_scaling["device"].str.contains("p520_max"))]["norm_flops"]  /  df_strong[(df_strong["device"].str.contains("p520_max")) & (df_strong["n"] == 1)]["norm_flops"].values[0], color="C2", alpha=0.6, width=0.2)
    hpl.bar(t+0.0, df_scaling[(df_scaling["m"] == np.ceil(48/t)) & (df_scaling["size"]/np.ceil(48/t) == 512) & (df_scaling["device"].str.contains("p520_hpc"))]["norm_flops"]  /  df_strong[(df_strong["device"].str.contains("p520_hpc")) & (df_strong["n"] == 1)]["norm_flops"].values[0], color="C1", alpha=0.6, width=0.2)


hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("Speedup per FPGA")
#hpl.set_ylim((0.0,72.0))
hpl.legend(loc="lower left", bbox_to_anchor=(0,0))
fig.savefig("hpl_torus_norm_2.pdf")

In [ ]:
fig, hpl = plt.subplots(1)
hpl.scatter(np.sqrt(df_strong["n"]), df_strong["lu_gflops"] * 1.0e-3, label="FPGA (SP) m=48", marker="x")
#hpl.scatter(np.sqrt(df_weak["n"]), df_weak["lu_gflops"] * 1.0e-3, label="FPGA (SP) m=48 ⋅ t", marker="x")
hpl.scatter(np.sqrt(df_cpu_strong["n"]), df_cpu_strong["lu_gflops"] * 1.0e-3, label="CPU (DP)  m=48", marker="x")
#hpl.scatter(np.sqrt(df_cpu_weak["n"]), df_cpu_weak["lu_gflops"] * 1.0e-3, label="CPU (DP)  m=48 ⋅ t", marker="x")
#hpl.scatter([3,4],[7.74884,13.5912], color="red", marker="x", label="Not validated")
#hpl.plot(np.linspace(0,5), [perf_last(48,n) * 1.0e-3 for n in np.linspace(0,5)], label="Model")
single_perf = df_strong[df_strong["N"] == 1]["lu_gflops"] * 1.0e-3
single_perf_cpu = df_cpu_strong[df_cpu_strong["N"] == 1]["lu_gflops"] * 1.0e-3
#hpl.step(list(range(1,7)), [0,single_perf, 7.06101e-01 * 4, 4.83296e-01 * 9, 3.68730e-01*16, 3.12129e-01*25], label="Strong Scaling")
hpl.plot(np.linspace(1,6), [single_perf * n * n  for n in np.linspace(1,6)], label="Extrapolation FPGA")
hpl.plot(np.linspace(1,6), [single_perf_cpu * n * n for n in np.linspace(1,6)], label="Extrapolation CPU")
#hpl.annotate("t² ⋅ %3.1f GFLOP/s" % (single_perf * 1.0e3), (2.2,17))
#hpl.annotate("t² ⋅ p_t", (3.8,7))
hpl.set_xlabel("Torus Width (t)")
hpl.set_ylabel("TFLOP/s")
hpl.set_xlim((0,6))
hpl.set_ylim((0,20))
hpl.legend()
fig.savefig("hpl_torus_strong.pdf")